In [ ]:
!pip install datasets
!pip install transformers
!pip install einops accelerate bitsandbytes
!pip install sentence_transformers
!pip install git+https://github.com/huggingface/peft.git
!pip install accelerate
!pip install evaluate
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.7 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=78182167708b76d25d9a5e6535101bb082dd803b6d93a716cad78e4f0e3b6ff5
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built 

In [ ]:
!pip install --upgrade peft
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 60.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [ ]:
MAX_LEN = 300
llm_model = 'recogna-nlp/bode-7b-alpaca-pt-br'
hf_auth = 'you_key'

## Data preparation

In [ ]:
from datasets import load_dataset
dataset = load_dataset("Silly-Machine/TuPyE-Dataset",'binary')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

### Target distribution in the train dataset

In [ ]:
pos_weights = len(dataset['train'].to_pandas()) / (2 * dataset['train'].to_pandas().hate.value_counts()[1])
neg_weights = len(dataset['train'].to_pandas()) / (2 * dataset['train'].to_pandas().hate.value_counts()[0])


### Compute the maximum length of the column text

In [ ]:
# Number of Characters
max_char = dataset['train'].to_pandas()['text'].str.len().max()
# Number of Words
max_words = dataset['train'].to_pandas()['text'].str.split().str.len().max()


### Let's take a look to one row example of training data

In [ ]:
dataset['train'][0]

{'source': 'twitter',
 'text': 'rt @user quero mais aaaaaa ',
 'researcher': 'leite et al',
 'year': 2020,
 'aggressive': 0,
 'hate': 0}

## Data Processing

In [ ]:
col_to_delete = ['source','researcher','year','aggressive']

# Load Llama 2 Tokenizer
from transformers import AutoTokenizer, DataCollatorWithPadding
from peft import PeftModel, PeftConfig
#
config = PeftConfig.from_pretrained(llm_model)

llama_tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path,add_prefix_space=True,trust_remote_code=True, token=hf_auth)
llama_tokenizer.pad_token_id = llama_tokenizer.eos_token_id
llama_tokenizer.pad_token = llama_tokenizer.eos_token

def llama_preprocessing_function(examples):
    return llama_tokenizer(examples['text'], truncation=True, max_length=MAX_LEN)

llama_tokenized_datasets = dataset.map(llama_preprocessing_function, batched=True, remove_columns=col_to_delete)
llama_tokenized_datasets = llama_tokenized_datasets.rename_column("hate", "label")
llama_tokenized_datasets.set_format("torch")

# Data collator for padding a batch of examples to the maximum length seen in the batch
llama_data_collator = DataCollatorWithPadding(tokenizer=llama_tokenizer)

adapter_config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Map:   0%|          | 0/34934 [00:00<?, ? examples/s]

Map:   0%|          | 0/8734 [00:00<?, ? examples/s]

## Models

In [ ]:
from transformers import AutoModelForSequenceClassification
import torch

llama_model =  AutoModelForSequenceClassification.from_pretrained(
  pretrained_model_name_or_path=config.base_model_name_or_path,
  num_labels=2,
  device_map="auto",
  offload_folder="offload",
  trust_remote_code=True,
  token=hf_auth
)

llama_model.config.pad_token_id = llama_model.config.eos_token_id

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-2-7b-chat-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### LoRa setup for Llama 2 classifier

In [ ]:
from peft import get_peft_model, LoraConfig, TaskType
llama_peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, r=16, lora_alpha=16, lora_dropout=0.05, bias="none",
    target_modules=[
        "q_proj",
        "v_proj",
    ],
)

llama_model = get_peft_model(llama_model, llama_peft_config)
llama_model.print_trainable_parameters()

trainable params: 8,396,800 || all params: 6,615,748,608 || trainable%: 0.12692138860666938


## Setup the trainer

### Evaluation Metrics

In [ ]:
# import evaluate
# import numpy as np

# def compute_metrics(eval_pred):
#     # All metrics are already predefined in the HF `evaluate` package
#     precision_metric = evaluate.load("precision")
#     recall_metric = evaluate.load("recall")
#     f1_metric= evaluate.load("f1")
#     accuracy_metric = evaluate.load("accuracy")

#     logits, labels = eval_pred # eval_pred is the tuple of predictions and labels returned by the model
#     predictions = np.argmax(logits, axis=-1)
#     precision = precision_metric.compute(predictions=predictions, references=labels)["precision"]
#     recall = recall_metric.compute(predictions=predictions, references=labels)["recall"]
#     f1 = f1_metric.compute(predictions=predictions, references=labels)["f1"]
#     accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
#     # The trainer is expecting a dictionary where the keys are the metrics names and the values are the scores.
#     return {"precision": precision, "recall": recall, "f1-score": f1, 'accuracy': accuracy}

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
# Custom function to compute metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Weighted metrics
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)

    return {
        'accuracy': acc,
        'f1_weighted': f1,
        'precision_weighted': precision,
        'recall_weighted': recall
    }

### Custom Trainer for Weighted Loss

In [ ]:
from transformers import Trainer

class WeightedCELossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        # Get model's predictions
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # Compute custom loss
        loss_fct = torch.nn.CrossEntropyLoss(weight=torch.tensor([neg_weights, pos_weights], device=model.device, dtype=logits.dtype))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss


In [ ]:
from transformers import TrainingArguments, Trainer

llama_model = llama_model.cuda()

lr = 1e-5
batch_size = 8
num_epochs = 20
training_args = TrainingArguments(
    output_dir="llama-lora-token-classification",
    learning_rate=lr,
    lr_scheduler_type= "constant",
    warmup_ratio= 0.1,
    warmup_steps=500,
    max_grad_norm= 0.3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.001,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="wandb",
    fp16=True,
    gradient_checkpointing=True,
)



llama_trainer = WeightedCELossTrainer(
    model=llama_model,
    args=training_args,
    train_dataset=llama_tokenized_datasets['train'],
    eval_dataset=llama_tokenized_datasets["test"],
    data_collator=llama_data_collator,
    compute_metrics=compute_metrics
)


In [ ]:
# 7 - Train the model
llama_trainer.train()

# 8 - Save the fine-tuned model
llama_model.save_pretrained('./fine_tuned_custom_sentiment_model')
llama_tokenizer.save_pretrained('./fine_tuned_custom_sentiment_model')


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Weighted,Precision Weighted,Recall Weighted
1,0.763300,0.791161,0.804900,0.810975,0.817558,0.804900
2,0.714100,0.696034,0.779711,0.801797,0.832187,0.779711
3,0.648800,0.700089,0.814174,0.823956,0.835610,0.814174
4,0.633000,0.651622,0.772155,0.799864,0.842690,0.772155
5,0.629000,0.652567,0.801580,0.818989,0.842807,0.801580
6,0.654400,0.672130,0.826883,0.833827,0.841866,0.826883
7,0.597000,0.657039,0.819785,0.830292,0.843296,0.819785
8,0.634400,0.693130,0.840966,0.842275,0.843626,0.840966
9,0.648100,0.740680,0.857339,0.850309,0.844623,0.857339
10,0.632300,0.668333,0.836272,0.840225,0.844563,0.836272


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs fo

('./fine_tuned_custom_sentiment_model/tokenizer_config.json',
 './fine_tuned_custom_sentiment_model/special_tokens_map.json',
 './fine_tuned_custom_sentiment_model/tokenizer.model',
 './fine_tuned_custom_sentiment_model/added_tokens.json',
 './fine_tuned_custom_sentiment_model/tokenizer.json')

In [ ]:
results = llama_trainer.evaluate()
print(results)

{'eval_loss': 0.6418256163597107, 'eval_accuracy': 0.8219601557133043, 'eval_f1_weighted': 0.8334531410324304, 'eval_precision_weighted': 0.8482025966621025, 'eval_recall_weighted': 0.8219601557133043, 'eval_runtime': 111.1117, 'eval_samples_per_second': 78.606, 'eval_steps_per_second': 9.828, 'epoch': 20.0}


## Upload fine tuned model

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' creden

In [ ]:
refined_model_name = "Silly-Machine/TuPy-Llama-Lora-Binary-Classifier_e20"
# Load the model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained('./fine_tuned_custom_sentiment_model')
tokenizer = AutoTokenizer.from_pretrained('./fine_tuned_custom_sentiment_model')

# Push to Hugging Face
model.push_to_hub(refined_model_name)
tokenizer.push_to_hub(refined_model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-2-7b-chat-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/integrations/peft.py:391: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Silly-Machine/TuPy-Llama-Lora-Binary-Classifier_e20/commit/b2e74d16ee455299d15c8ae8a02848fe25e085ba', commit_message='Upload tokenizer', commit_description='', oid='b2e74d16ee455299d15c8ae8a02848fe25e085ba', pr_url=None, pr_revision=None, pr_num=None)